## 세티즌 데이터 확인용
#### 태블릿 크롤링 - 이름, 저장공간, 제품코드, 설명, 상태, 가격, 시간, url, 판매여부
##### 0420

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36'}

tablet = []
name = []
storage = []
product_code = []
memo = []
condition = []
price = []
upload_time = []
sold = []
url_info = []
site = []
category = []



#마지막 페이지까지 크롤링 반복
for i in range(1,18):
    url = f"https://market.cetizen.com/market.php?p=1&keyword_p=&pno=&just_one=&just_one_name=&product_no=&pw=&pm=&m_no=&p_title=&pname=&p_key=&p_modelno=&p_model=&storage=&p_cat=&pm_modal=&series_modal=&datamode=&q=&at=&section=&cat%5B1%5D=2&price_range=0%3B5000000&auc_price1=0&auc_price2=5%2C000%2C000&storage_range=0%3B1024&storage1=0&storage2=1024&escrow_motion=3&select_arrange=headnum&desc=+&remove_dup=&list_type=1&p={i}"
    r = requests.get(url, headers=headers)
    r.status_code

    soup = BeautifulSoup(r.text,'html.parser')


    # 한페이지 정보 가져와 smartphone에 저장
    tablet = soup.find_all('div', class_= 'col-10')
    tablet = tablet[0::2]

    
    # 각 정보 해당 리스트에 저장하기
    for info in tablet:
        
        # 05/25 나오면 중단
        time_info = info.find("span", class_='gray2').text
        if (time_info == '05-25'): break
        
        # 상품 이름 
        name_info = info.find("span", class_='fsize6').text
        name.append(name_info)

        # 저장공간 정보는 정보 유무 판단 후, 정보 없는 경우 '정보없음'으로 저장
        if( info.find("span", class_='bg-gray3') ):
            storage.append(info.find("span", class_='bg-gray3').text)
        else : storage.append("정보없음")
    
        # 상품 코드 
        code_info = info.find("span", class_='ls-0').text
        product_code.append(code_info)
        
        # 상품 설명 
        memo_info = info.find("span", class_='pointer').text
        memo.append(memo_info)
        
        # 상품 상태 
        condition_info = info.find(['div','span'], class_= ['box_noicon_red', 'box_noicon_on']).text
        condition.append(condition_info)

        # 가격 정보는 삭제된(==거래완료된) 정보일 경우와 아닌 경우로 구분하여 저장
        if ( info.select("span > del") ) : 
            price.append(info.find("del").text)
        elif ( info.find("span", class_='black0')) : 
            price.append(info.find("span", class_='black0').text)
        else : 
            price.append( info.find_all("span", class_='gray1')[1].text )
        
        # 업로드 시간 (시간으로 표시된 정보는 오늘 날짜로 저장)
        time_info = info.find("span", class_='gray2').text
        if(':' in time_info) : 
            upload_time.append('06-02')
        elif('전' in time_info) :
            upload_time.append('06-02')
        else : upload_time.append(time_info)
        
        # 거래 완료 
        if ( info.find("span" , class_= "ml-2")) : 
            sold.append("거래완료")
        else : sold.append("-")
            
        # url
        url_add = 'https://market.cetizen.com/' + info.find('a')['href']
        url_info.append(url_add)
            
        # 사이트명과 카테고리
        site.append("cetizen")
        category.append("태블릿")

        
   # time.sleep(1)


In [4]:
# DataFrame 생성 및 확인

df = pd.DataFrame(
        {"사이트" : site, 
         "카테고리" : category,
         "판매여부" : sold,
         "이름" : name,
         "저장공간" : storage,
         "제품코드" : product_code,
         "상태" : condition,
         "가격" : price,
         "업로드시간" : upload_time,
         "설명" : memo,
         "url" : url_info
        }
    )

df.shape
df


,사이트,카테고리,판매여부,이름,저장공간,제품코드,상태,가격,업로드시간,설명,url
0,cetizen,태블릿,-,아이패드미니5 (2019) 셀룰러,정보없음,A2124,상급,"450,000",06-02,아이패드 미니5세대 풀박스 + 로지텍펜슬 판매!,https://market.cetizen.com//market.php?q=view&...
1,cetizen,태블릿,-,아이패드프로 11 (2021),1TB,A2377,새제품,"1,760,000",06-02,아이패드프로5세대 셀룰러 11인치 1TB,https://market.cetizen.com//market.php?q=view&...
2,cetizen,태블릿,-,갤럭시 탭 A7 LTE,64GB,SM-T505N,새제품,"270,000",06-01,갤럭시 탭A7 (T505) 다크그레이 64기가 미개봉 팔아요~,https://market.cetizen.com//market.php?q=view&...
3,cetizen,태블릿,-,갤럭시탭A8 10.5 LTE,정보없음,SM-X205N,새제품,"290,000",06-01,갤럭시탭A8(X205) 그레이 64기가 미개봉 팝니다.,https://market.cetizen.com//market.php?q=view&...
4,cetizen,태블릿,-,갤럭시 탭 S6라이트,64GB,SM-P610,상급,"300,000",06-01,갤럭시탭S6 라이트 블랙 64기가 특A급 팝니다.,https://market.cetizen.com//market.php?q=view&...
...,...,...,...,...,...,...,...,...,...,...,...
385,cetizen,태블릿,거래완료,갤럭시 노트 10.1 2014,32GB,SM-P600 WiFi,중급,"75,000",05-19,갤럭시탭10.1 2014에디션 16g,https://market.cetizen.com//market.php?q=view&...
386,cetizen,태블릿,거래완료,아이패드 에어 2 (2014),16GB,A1566,상급,"190,000",05-19,아이패드 에어2 실버 16G 셀룰러,https://market.cetizen.com//market.php?q=view&...
387,cetizen,태블릿,거래완료,비와이패드2,정보없음,BTV-DL09,새제품,"139,000",05-19,비와이패드2 완박 미개봉(정상해지),https://market.cetizen.com//market.php?q=view&...
388,cetizen,태블릿,거래완료,아이패드에어5,64GB,A2589,새제품,"679,000",05-19,아이패드에어5세대 미개봉 판매합니다(스페이스그레이),https://market.cetizen.com//market.php?q=view&...


In [5]:
# df.to_csv('cetizendata_tablet', index=False)
df.to_excel("cetizendata_tablet_0602.xlsx")